In [6]:
!pip install flaml
!pip install catboost
!pip install django
#!pip install prophet
!pip install statsmodels==0.12.1
!pip install bokeh
!pip install transformers
!pip install datasets
!pip install pyyaml==5.4.1

In [7]:
import pandas as pd 
import statsmodels 
import transformers
import datasets
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from django.conf import settings

In [8]:
# Load in and prepare data
from sklearn.datasets import load_iris
dataset = load_iris()
x = dataset.data
y = dataset.target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size = 0.2, random_state = 100)

In [70]:
# Function for visualizations of all estimators accuracy over time

#kn_param_list =[]
def interVisClass(x, y, t):
  # Import packages
  from flaml import AutoML
  from flaml.data import get_output_from_log
  import matplotlib.pyplot as plt
  from bokeh.layouts import row
  from bokeh.plotting import figure
  from bokeh.io import output_file, show, output_notebook
  from bokeh.layouts import gridplot
  from bokeh.models import Title, HoverTool
  from bokeh.plotting import ColumnDataSource
  import numpy as np 

  # Log Creation

  automl = AutoML()

  automl.fit(x, y, 
               task=t,
               log_type = 'best',
               estimator_list = ['lgbm'],
               log_file_name = 'iter_lgbm.log') #lgbm log
  lgbm_best = automl.best_config

  automl.fit(x, y, 
               task=t,
               log_type = 'best',
               estimator_list = ['xgboost'],
               log_file_name = 'iter_xgboost.log') #xgb log
  xgb_best = automl.best_config

  automl.fit(x, y, 
               task=t,
               log_type = 'best',
               estimator_list = ['rf'],
               log_file_name = 'iter_rf.log') #rf log
  rf_best = automl.best_config

  automl.fit(x, y, 
                task=t,
                log_type = 'best',
                estimator_list = ['extra_tree'],
                log_file_name = 'iter_et.log') #extra tree log
  et_best = automl.best_config
  
  automl.fit(x, y, 
                task=t,
                log_type = 'best',
                estimator_list = ['lrl1'],
                log_file_name = 'iter_lrl1.log') #Logistic Regression with L1 regularization log
  lrl1_best = automl.best_config
  
  automl.fit(x, y, 
                task=t,
                log_type = 'best',
                estimator_list = ['lrl2'],
                log_file_name = 'iter_lrl2.log') #Logistic Regression with L2 regularization log  
  lrl2_best = automl.best_config

  automl.fit(x, y, 
                task=t,
                log_type = 'best',
                estimator_list = ['catboost'],
                log_file_name = 'iter_catboost.log') #catboost log
  cat_best = automl.best_config

  automl.fit(x, y, 
                task=t,
                log_type = 'best',
                estimator_list = ['kneighbor'],
                log_file_name = 'iter_kn.log') #kneighbor log
  kn_best = automl.best_config


  # Output retreival

  time_history_lgbm, best_valid_loss_history_lgbm, valid_loss_history_lgbm, config_history_lgbm, metric_history_lgbm = \
      get_output_from_log(filename="iter_lgbm.log", time_budget=60) #lgbm output

  time_history_xgb, best_valid_loss_history_xgb, valid_loss_history_xgb, config_history_xgb, metric_history_xgb = \
      get_output_from_log(filename="iter_xgboost.log", time_budget=60) #xgb output

  time_history_rf, best_valid_loss_history_rf, valid_loss_history_rf, config_history_rf, metric_history_rf = \
      get_output_from_log(filename="iter_rf.log", time_budget=60) #rf output     

  time_history_et, best_valid_loss_history_et, valid_loss_history_et, config_history_et, metric_history_et = \
      get_output_from_log(filename="iter_et.log", time_budget=60)  #extra tree output     

  time_history_lrl1, best_valid_loss_history_lrl1, valid_loss_history_lrl1, config_history_lrl1, metric_history_lrl1 = \
      get_output_from_log(filename="iter_lrl1.log", time_budget=60)  #Logistic Regression with L1 regularization output 

  time_history_lrl2, best_valid_loss_history_lrl2, valid_loss_history_lrl2, config_history_lrl2, metric_history_lrl2 = \
      get_output_from_log(filename="iter_lrl2.log", time_budget=60)  #Logistic Regression with L2 regularization output 

  time_history_cat, best_valid_loss_history_cat, valid_loss_history_cat, config_history_cat, metric_history_cat = \
      get_output_from_log(filename="iter_catboost.log", time_budget=60)  #catboost output 

  time_history_kn, best_valid_loss_history_kn, valid_loss_history_kn, config_history_kn, metric_history_kn = \
      get_output_from_log(filename="iter_kn.log", time_budget=60)  #kneighbor output 

  # Hyper-parameter dataframe 

  count = 0
  lgbm_param_list =[]
  for i in config_history_lgbm:
    params = config_history_lgbm[count]['Current Hyper-parameters']
    lgbm_param_list.append(params)
    count+=1
  lgbm_df = pd.DataFrame(lgbm_param_list)
  lgbm_df.insert(0, 'Time', time_history_lgbm)

  count = 0
  xgb_param_list =[]
  for i in config_history_xgb:
    params = config_history_xgb[count]['Current Hyper-parameters']
    xgb_param_list.append(params)
    count+=1
  xgb_df = pd.DataFrame(xgb_param_list)
  xgb_df.insert(0, 'Time', time_history_xgb)

  count = 0
  rf_param_list =[]
  for i in config_history_rf:
    params = config_history_rf[count]['Current Hyper-parameters']
    rf_param_list.append(params)
    count+=1
  rf_df = pd.DataFrame(rf_param_list)
  rf_df.insert(0, 'Time', time_history_rf)
  
  count = 0
  et_param_list =[]
  for i in config_history_et:
    params = config_history_et[count]['Current Hyper-parameters']
    et_param_list.append(params)
    count+=1
  et_df = pd.DataFrame(et_param_list)
  et_df.insert(0, 'Time', time_history_et)
  
  count = 0
  lrl1_param_list =[]
  for i in config_history_lrl1:
    params = config_history_lrl1[count]['Current Hyper-parameters']
    lrl1_param_list.append(params)
    count+=1
  lrl1_df = pd.DataFrame(lrl1_param_list)
  lrl1_df.insert(0, 'Time', time_history_lrl1)
  
  count = 0
  lrl2_param_list =[]
  for i in config_history_lrl2:
    params = config_history_lrl2[count]['Current Hyper-parameters']
    lrl2_param_list.append(params)
    count+=1
  lrl2_df = pd.DataFrame(lrl2_param_list)
  lrl2_df.insert(0, 'Time', time_history_lrl2)

  count = 0
  cat_param_list =[]
  for i in config_history_cat:
    params = config_history_cat[count]['Current Hyper-parameters']
    cat_param_list.append(params)
    count+=1
  cat_df = pd.DataFrame(cat_param_list)
  cat_df.insert(0, 'Time', time_history_cat)

  count = 0
  kn_param_list =[]
  for i in config_history_kn:
    params = config_history_kn[count]['Current Hyper-parameters']
    kn_param_list.append(params)
    count+=1
  kn_df = pd.DataFrame(kn_param_list)
  kn_df.insert(0, 'Time', time_history_kn)

  # Line chart showing classification models' wall clock time vs validation accuracy
  
  output_notebook()

  x_values1 = time_history_lgbm
  y_values1 = 1 - np.array(best_valid_loss_history_lgbm)

  x_values2 = time_history_xgb
  y_values2 = 1-np.array(best_valid_loss_history_xgb)

  x_values3 = time_history_rf
  y_values3 = 1-np.array(best_valid_loss_history_rf)
  
  x_values4 = time_history_et
  y_values4 = 1-np.array(best_valid_loss_history_et)
  
  x_values5 = time_history_lrl1
  y_values5 = 1-np.array(best_valid_loss_history_lrl1)

  x_values6 = time_history_lrl2
  y_values6 = 1-np.array(best_valid_loss_history_lrl2)

  x_values7 = time_history_cat
  y_values7 = 1-np.array(best_valid_loss_history_cat)

  x_values8 = time_history_kn
  y_values8 = 1-np.array(best_valid_loss_history_kn)

  TOOLTIPS = [
              ("Time(s)", "$x"),
              ("Accuracy", "$y"),            
              
  ]
  p = figure(title = "Classification Model Accuracies", tooltips=TOOLTIPS)
  #p.add_tools(HoverTool())

  p.add_layout(Title(text="Wall Clock Time (s)", align="center"), "below")
  p.add_layout(Title(text="Validation Accuracy", align="center"), "left")

  p.circle(x=x_values1, y=y_values1, size=3, legend_label="LGBM", color ="blue")
  p.step(x=x_values1, y=y_values1, legend_label="LGBM", color ="blue")

  p.circle(x=x_values2, y=y_values2, size=3, legend_label="XGBoost", color="green")
  p.step(x=x_values2, y=y_values2, legend_label="XGBoost", color="green")

  p.circle(x=x_values3, y=y_values3, size=3, legend_label="Random Forest", color="purple")
  p.step(x=x_values3, y=y_values3, legend_label="Random Forest", color="purple")
  
  p.circle(x=x_values4, y=y_values4, size=3, legend_label="Extra Tree", color="orange")
  p.step(x=x_values4, y=y_values4, legend_label="Extra Tree", color="orange")
  
  p.circle(x=x_values5, y=y_values5, size=3, legend_label="LRL1", color="black")
  p.step(x=x_values5, y=y_values5, legend_label="LRL1", color="black")
  p.circle(x=x_values6, y=y_values6, size=3, legend_label="LRL2", color="grey")
  p.step(x=x_values6, y=y_values6, legend_label="LRL2", color="grey")

  p.circle(x=x_values7, y=y_values7, size=3, legend_label="Catboost", color="brown")
  p.step(x=x_values7, y=y_values7, legend_label="Catboost", color="brown")

  p.circle(x=x_values8, y=y_values8, size=3, legend_label="k-Neighbor", color="red")
  p.step(x=x_values8, y=y_values8, legend_label="k-Neighbor", color="red")


  p.legend.location = "bottom_right"
  p.legend.click_policy="hide"


  # Output for datatables showing best hyperparameters
  show(p)
  print('' '\n')
  ########################### LGBM Charts###########################################
  display(lgbm_df.style.set_caption("LGBM Best Hyper-parameters"))
  print('' '\n')

  # Creates list of hyperparameters for LGBM
  lgbmh_list = []
  for index in range(len(lgbm_param_list)):
    for key in lgbm_param_list[index]:
        lgbmh_list.append(lgbm_param_list[index][key])

  # Separate hyperparameters
  n_estimators_list = []
  i = 0
  while i < len(lgbmh_list):
    n_estimators_list.append(lgbmh_list[i])
    i += 8

  num_leaves_list = []
  i = 1
  while i < len(lgbmh_list):
    num_leaves_list.append(lgbmh_list[i])
    i += 8

  min_child_samples_list = []
  i = 2
  while i < len(lgbmh_list):
    min_child_samples_list.append(lgbmh_list[i])
    i += 8

  # Create step chart of hyperparameters
  n_estimators_values = n_estimators_list
  lgbmy_values1 = 1 - np.array(best_valid_loss_history_lgbm)
  num_leaves_values = num_leaves_list
  lgbmy_values2 = 1 - np.array(best_valid_loss_history_lgbm)
  min_child_samples_values = min_child_samples_list
  lgbmy_values3 = 1 - np.array(best_valid_loss_history_lgbm)

  # Titles and tooltips for chart
  tooltips_lgbm = [("Hyperparameter value", "$x"), ("Accuracy", "$y")]
  lgbm_chart = figure(title = "Hyperparameters for LGBM", tooltips=tooltips_lgbm)
  lgbm_chart.add_layout(Title(text="Hyperparameters", align="center"), "below")
  lgbm_chart.add_layout(Title(text="Validation Accuracy", align="center"), "left")

  # Creates circles and steps for chart
  lgbm_chart.circle(x=n_estimators_values, y=lgbmy_values1, size=5, legend_label="n Estimators", color ="blue")
  lgbm_chart.circle(x= num_leaves_values, y=lgbmy_values2, size=5, legend_label="Number of Leaves", color="green")
  lgbm_chart.circle(x=min_child_samples_values, y=lgbmy_values3, size=5, legend_label="Min Child Samples", color ="orange")

  # Creates legend
  lgbm_chart.legend.location = "bottom_right"
  lgbm_chart.legend.click_policy="hide"
  show(lgbm_chart)

  ########################### XGBoost Charts###########################################
  display(xgb_df.style.set_caption("XGBoost Best Hyper-parameters"))
  print('' '\n')

  # Creates list of hyperparameters for XGBoost
  xgb_list = []
  for index in range(len(xgb_param_list)):
    for key in xgb_param_list[index]:
        xgb_list.append(xgb_param_list[index][key])
  
  # Separate hyperparameters
  xn_estimators_list = []
  i = 0
  while i < len(xgb_list):
    xn_estimators_list.append(xgb_list[i])
    i += 9

  xnum_leaves_list = []
  i = 1
  while i < len(xgb_list):
    xnum_leaves_list.append(xgb_list[i])
    i += 9

  xmin_child_samples_list = []
  i = 2
  while i < len(xgb_list):
    xmin_child_samples_list.append(xgb_list[i])
    i += 9

  # Create step chart of hyperparameters
  xn_estimators_values = xn_estimators_list
  xgby_values1 = 1 - np.array(best_valid_loss_history_xgb)
  xnum_leaves_values = xnum_leaves_list
  xgby_values2 = 1 - np.array(best_valid_loss_history_xgb)
  xmin_child_samples_values = xmin_child_samples_list
  xgby_values3 = 1 - np.array(best_valid_loss_history_xgb)

  # Titles and tooltips for chart
  tooltips_xgb = [("Hyperparameter value", "$x"), ("Accuracy", "$y")]
  xgb_chart = figure(title = "Hyperparameters for XGBoost", tooltips=tooltips_xgb)
  xgb_chart.add_layout(Title(text="Hyperparameters", align="center"), "below")
  xgb_chart.add_layout(Title(text="Validation Accuracy", align="center"), "left")

  # Creates circles and steps for chart
  xgb_chart.circle(x=xn_estimators_values, y=xgby_values1, size=5, legend_label="n Estimators", color ="blue")
  #lgbm_chart.step(x=n_estimators_values, y=lgbmy_values1, legend_label="n Estimators", color ="blue")
  xgb_chart.circle(x= xnum_leaves_values, y=xgby_values2, size=5, legend_label="Number of Leaves", color="green")
  #lgbm_chart.step(x= num_leaves_values, y=lgbmy_values2, legend_label="Number of Leaves", color="green")
  xgb_chart.circle(x=xmin_child_samples_values, y=xgby_values3, size=5, legend_label="Min Child Samples", color ="orange")
  #lgbm_chart.step(x=min_child_samples_values, y=lgbmy_values3, legend_label="Min Child Samples", color ="orange")

  # Creates legend
  xgb_chart.legend.location = "bottom_right"
  xgb_chart.legend.click_policy="hide"
  show(xgb_chart)
  
  ########################### Random Forest Charts###########################################
  display(rf_df.style.set_caption("Random Forest Best Hyper-parameters"))
  print('' '\n')

  # Creates list of hyperparameters for XGBoost
  rf_list = []
  for index in range(len(rf_param_list)):
    for key in rf_param_list[index]:
        rf_list.append(rf_param_list[index][key])
  
  # Separate hyperparameters
  rfn_estimators_list = []
  i = 0
  while i < len(rf_list):
    rfn_estimators_list.append(rf_list[i])
    i += 4

  rfmax_features_list = []
  i = 1
  while i < len(rf_list):
    rfmax_features_list.append(rf_list[i])
    i += 4

  rfmax_leaves_list = []
  i = 2
  while i < len(rf_list):
    rfmax_leaves_list.append(rf_list[i])
    i += 4

  # Create step chart of hyperparameters
  rfn_estimators_values = rfn_estimators_list
  rfy_values1 = 1 - np.array(best_valid_loss_history_rf)
  rfmax_features_values = rfmax_features_list
  rfy_values2 = 1 - np.array(best_valid_loss_history_rf)
  rfmax_leaves_values =  rfmax_leaves_list
  rfy_values3 = 1 - np.array(best_valid_loss_history_rf)

  # Titles and tooltips for chart
  tooltips_rf = [("Hyperparameter value", "$x"), ("Accuracy", "$y")]
  rf_chart = figure(title = "Hyperparameters for Random Forest", tooltips=tooltips_rf)
  rf_chart.add_layout(Title(text="Hyperparameters", align="center"), "below")
  rf_chart.add_layout(Title(text="Validation Accuracy", align="center"), "left")

  # Creates circles and steps for chart
  rf_chart.circle(x=rfn_estimators_values, y=rfy_values1, size=5, legend_label="n Estimators", color ="blue")
  rf_chart.circle(x=rfmax_features_values, y=rfy_values2, size=5, legend_label="Max Features", color="green")
  rf_chart.circle(x=rfmax_leaves_values, y=rfy_values3, size=5, legend_label="Max Leaves", color ="orange")

  # Creates legend
  rf_chart.legend.location = "bottom_right"
  rf_chart.legend.click_policy="hide"
  show(rf_chart)

  ########################### Extra Tree Charts###########################################
  display(et_df.style.set_caption("Extra Tree Best Hyper-parameters"))
  print('' '\n')

  # Creates list of hyperparameters for Extra Tree
  et_list = []
  for index in range(len(et_param_list)):
    for key in et_param_list[index]:
        et_list.append(et_param_list[index][key])
  
  # Separate hyperparameters
  etn_estimators_list = []
  i = 0
  while i < len(et_list):
    etn_estimators_list.append(et_list[i])
    i += 4

  etmax_features_list = []
  i = 1
  while i < len(et_list):
    etmax_features_list.append(et_list[i])
    i += 4

  etmax_leaves_list = []
  i = 2
  while i < len(et_list):
    etmax_leaves_list.append(et_list[i])
    i += 4

  # Create step chart of hyperparameters
  etn_estimators_values = etn_estimators_list
  ety_values1 = 1 - np.array(best_valid_loss_history_et)
  etmax_features_values = etmax_features_list
  ety_values2 = 1 - np.array(best_valid_loss_history_et)
  etmax_leaves_values =  etmax_leaves_list
  ety_values3 = 1 - np.array(best_valid_loss_history_et)

  # Titles and tooltips for chart
  tooltips_et = [("Hyperparameter value", "$x"), ("Accuracy", "$y")]
  et_chart = figure(title = "Hyperparameters for Extra Tree", tooltips=tooltips_et)
  et_chart.add_layout(Title(text="Hyperparameters", align="center"), "below")
  et_chart.add_layout(Title(text="Validation Accuracy", align="center"), "left")

  # Creates circles and steps for chart
  et_chart.circle(x=etn_estimators_values, y=ety_values1, size=5, legend_label="n Estimators", color ="blue")
  et_chart.circle(x=etmax_features_values, y=ety_values2, size=5, legend_label="Max Features", color="green")
  et_chart.circle(x=etmax_leaves_values, y=ety_values3, size=5, legend_label="Max Leaves", color ="orange")

  # Creates legend
  et_chart.legend.location = "bottom_right"
  et_chart.legend.click_policy="hide"
  show(et_chart)

  ########################### LRL1 Charts###########################################
  display(lrl1_df.style.set_caption("LRL1 Best Hyper-parameters"))
  print('' '\n')

  # creates list for accuracy for LRL1
  lrl1_list = []
  for index in range(len(lrl1_param_list)):
    for key in lrl1_param_list[index]:
      lrl1_list.append(lrl1_param_list[index][key])
  
  # Scatter plot for LRL1 showing C vs accuracy
  lrl1_tooltips = [("C", "$x"), ("Accuracy", "$y")]
  lrl1_scatterplot = figure(title = "Hyperparameters of LRL1", tooltips=lrl1_tooltips)
  x_lrl1 = lrl1_list
  y_lrl1 = 1 - np.array(best_valid_loss_history_lrl1)
  lrl1_scatterplot.scatter(x_lrl1, y_lrl1, size=7)
  #lrl1_scatterplot.circle(x=x_lrl1, y=y_lrl1, size=7)
  lrl1_scatterplot.add_layout(Title(text="C Value", align="center"), "below")
  lrl1_scatterplot.add_layout(Title(text="Validation Accuracy", align="center"), "left")
  show(lrl1_scatterplot)

  ########################### LRL2 Charts###########################################
  display(lrl2_df.style.set_caption("LRL2 Best Hyper-parameters"))
  print('' '\n')

  # creates list for accuracy for LRL2
  lrl2_list = []
  for index in range(len(lrl2_param_list)):
    for key in lrl2_param_list[index]:
      lrl2_list.append(lrl2_param_list[index][key])
  
  # Scatter plot for LRL2 showing C vs accuracy
  lrl2_tooltips = [("C", "$x"), ("Accuracy", "$y")]
  lrl2_scatterplot = figure(title = "Hyperparameters of LRL2", tooltips=lrl2_tooltips)
  x = lrl2_list
  y = 1 - np.array(best_valid_loss_history_lrl2)
  lrl2_scatterplot.scatter(x, y)
  lrl2_scatterplot.circle(x=x, y=y, size=7)
  lrl2_scatterplot.add_layout(Title(text="C Value", align="center"), "below")
  lrl2_scatterplot.add_layout(Title(text="Validation Accuracy", align="center"), "left")
  show(lrl2_scatterplot)
  
  ########################### Catboost Charts###########################################
  display(cat_df.style.set_caption("Catboost Best Hyper-parameters"))
  print('' '\n')

  # Creates list of hyperparameters for Extra Tree
  cat_list = []
  for index in range(len(cat_param_list)):
    for key in cat_param_list[index]:
        cat_list.append(cat_param_list[index][key])
  
  # Separate hyperparameters
  cate_stop_list = []
  i = 0
  while i < len(cat_list):
    cate_stop_list.append(cat_list[i])
    i += 3

  catn_estimators_list = []
  i = 2
  while i < len(cat_list):
    catn_estimators_list.append(cat_list[i])
    i += 3

  # Create scatterplot of hyperparameters
  catn_estimators_values = catn_estimators_list
  caty_values1 = 1 - np.array(best_valid_loss_history_cat)
  cate_stop_values =  cate_stop_list
  caty_values3 = 1 - np.array(best_valid_loss_history_cat)

  # Titles and tooltips for chart
  tooltips_cat = [("Hyperparameter value", "$x"), ("Accuracy", "$y")]
  cat_chart = figure(title = "Hyperparameters for Catboost", tooltips=tooltips_cat)
  cat_chart.add_layout(Title(text="Hyperparameters", align="center"), "below")
  cat_chart.add_layout(Title(text="Validation Accuracy", align="center"), "left")

  # Creates circles and steps for chart
  cat_chart.circle(x=catn_estimators_values, y=caty_values1, size=5, legend_label="n Estimators", color ="blue")
  cat_chart.circle(x=cate_stop_values, y=caty_values3, size=5, legend_label="Early Stopping Rounds", color ="orange")

  # Creates legend
  cat_chart.legend.location = "bottom_right"
  cat_chart.legend.click_policy="hide"
  show(cat_chart)

  ########################### KNN Charts###########################################
  display(kn_df.style.set_caption("k-Neighbor Best Hyper-parameters"))

  # creates list for accuracy for KNN
  knn_list = []
  for index in range(len(kn_param_list)):
    for key in kn_param_list[index]:
      knn_list.append(kn_param_list[index][key])
  
  # Scatter plot for KNN showing n-neighbors vs accuracy
  KNN_tooltips = [("Number of Neighbors", "$x"), ("Accuracy", "$y")]
  KNN_scatterplot = figure(title = "Hyperparameters of KNN", tooltips=KNN_tooltips)
  x = knn_list
  y = 1 - np.array(best_valid_loss_history_kn)
  KNN_scatterplot.scatter(x, y)
  KNN_scatterplot.circle(x=x, y=y, size=7)
  KNN_scatterplot.add_layout(Title(text="Number of Neighbors", align="center"), "below")
  KNN_scatterplot.add_layout(Title(text="Validation Accuracy", align="center"), "left")
  show(KNN_scatterplot)
 

In [71]:
interVisClass(x_train, y_train, "classification")

Streaming output truncated to the last 5000 lines.
[flaml.automl: 04-28 01:50:01] {2567} INFO - iteration 1087, current learner lgbm
[flaml.automl: 04-28 01:50:01] {2750} INFO -  at 46.9s,	estimator lgbm's best error=0.1243,	best estimator lgbm's best error=0.1243
[flaml.automl: 04-28 01:50:01] {2567} INFO - iteration 1088, current learner lgbm
[flaml.automl: 04-28 01:50:02] {2750} INFO -  at 47.0s,	estimator lgbm's best error=0.1243,	best estimator lgbm's best error=0.1243
[flaml.automl: 04-28 01:50:02] {2567} INFO - iteration 1089, current learner lgbm
[flaml.automl: 04-28 01:50:02] {2750} INFO -  at 47.0s,	estimator lgbm's best error=0.1243,	best estimator lgbm's best error=0.1243
[flaml.automl: 04-28 01:50:02] {2567} INFO - iteration 1090, current learner lgbm
[flaml.automl: 04-28 01:50:02] {2750} INFO -  at 47.1s,	estimator lgbm's best error=0.1243,	best estimator lgbm's best error=0.1243
[flaml.automl: 04-28 01:50:02] {2567} INFO - iteration 1091, current learner lgbm
[flaml.auto

,Time,n_estimators,num_leaves,min_child_samples,learning_rate,log_max_bin,colsample_bytree,reg_alpha,reg_lambda
0,0.056518,4,4,20,0.100000,8,1.000000,0.000977,1.000000
1,0.138737,4,4,12,0.267705,7,1.000000,0.001348,1.444258
2,0.177354,6,4,9,0.726059,9,0.928500,0.003684,0.753248
3,0.744262,5,7,7,0.638611,10,1.000000,0.002083,1.414675
4,3.289457,4,6,6,0.768372,10,0.933441,0.000977,0.346083
5,13.570734,4,7,5,0.678558,9,0.980001,0.000977,0.413928
6,14.041190,4,8,5,0.675259,9,0.883632,0.002225,0.513885
7,25.831542,4,8,5,0.677247,10,0.892926,0.005419,0.430340


,Time,n_estimators,max_leaves,min_child_weight,learning_rate,subsample,colsample_bylevel,colsample_bytree,reg_alpha,reg_lambda
0,0.070694,4,4,1.000000,0.100000,1.000000,1.000000,1.000000,0.000977,1.000000
1,0.178393,4,4,0.262081,0.259125,0.926674,1.000000,1.000000,0.001393,0.180969
2,0.225252,4,4,1.863022,1.000000,0.851363,1.000000,0.946138,0.001831,0.279017
3,0.596087,4,4,1.292499,0.821069,0.817473,0.972262,0.999326,0.000977,0.055276
4,8.420996,4,4,0.874906,1.000000,0.873020,0.928843,0.818354,0.000977,0.007399
5,14.698796,4,4,1.446321,1.000000,0.944577,0.757480,0.954279,0.000977,0.002953
6,22.187457,5,4,1.869329,1.000000,0.872962,0.947040,0.978188,0.000977,0.015650


,Time,n_estimators,max_features,max_leaves,criterion
0,1.171984,4,0.500000,4,entropy
1,2.436383,4,0.484601,6,gini
2,7.383654,6,0.483105,5,gini
3,8.647476,4,0.484601,6,entropy
4,11.371930,4,0.570912,5,entropy
5,27.635355,4,0.601801,4,entropy
6,31.315732,4,0.712289,4,entropy
7,41.895058,4,0.509053,5,entropy
8,51.233324,6,0.722350,5,gini


,Time,n_estimators,max_features,max_leaves,criterion
0,1.171683,4,0.500000,4,entropy
1,2.435308,4,0.484601,6,gini
2,5.051527,5,0.319550,6,entropy
3,6.409475,4,0.484601,6,entropy
4,9.025945,4,0.486102,7,gini
5,11.757588,4,0.572680,6,gini
6,14.165704,4,0.523267,10,entropy
7,26.221264,4,0.808218,11,entropy
8,36.600335,5,0.951452,7,entropy


,Time,C
0,0.539733,1.000000
1,1.602863,4.000000
2,2.668162,64.000000
3,3.198590,1909.500372
4,5.849544,32209.012484


,Time,C
0,1.590989,1.000000
1,1.956397,4.000000
2,3.020437,28.411973
3,6.004717,6.902819
4,9.767023,21.412363
5,21.916798,10.059946
6,31.709675,18.928805


,Time,early_stopping_rounds,learning_rate,n_estimators
0,12.458165,10,0.100000,53
1,22.614959,10,0.062336,85
2,30.595466,12,0.100000,53
3,37.148932,15,0.066923,67
4,46.504518,13,0.107966,55
5,52.216945,17,0.159798,51
6,54.181342,25,0.158868,51


,Time,n_neighbors
0,0.546440,5
1,1.620126,8
2,5.337818,6
